In [1]:
import lsst.daf.butler as dafButler

repo = '/repo/main'
instrument = "LSSTComCam"
butler_11= dafButler.Butler(repo, collections='LSSTComCam/runs/DRP/DP1/w_2025_11/DM-49472', instrument=instrument)
butler_12= dafButler.Butler(repo, collections='LSSTComCam/runs/DRP/DP1/d_2025_03_25/DM-49693', instrument=instrument)
butler_13= dafButler.Butler(repo, collections='LSSTComCam/runs/DRP/DP1/w_2025_13/DM-49751', instrument=instrument)
butler_14= dafButler.Butler(repo, collections='LSSTComCam/runs/DRP/DP1/w_2025_14/DM-49864', instrument=instrument)
test_11 = butler_11.getURI('object_forced_source', dataId={'skymap':'lsst_cells_v1', 'tract': 10704, 'patch': 22})
test_12 = butler_11.getURI('object_forced_source', dataId={'skymap':'lsst_cells_v1', 'tract': 10704, 'patch': 22})
test_13 = butler_13.getURI('object_forced_source', dataId={'skymap':'lsst_cells_v1', 'tract': 10704, 'patch': 22})
test_14 = butler_14.getURI('object_forced_source', dataId={'skymap':'lsst_cells_v1', 'tract': 10704, 'patch': 22})
print(test_11)
print(test_12)
print(test_13)
print(test_14)
size_mb = test_11.size() / (1024 ** 2)
print(f"Size in w11: {size_mb:.2f} MB")
size_mb = test_12.size() / (1024 ** 2)
print(f"Size in w12: {size_mb:.2f} MB")
size_mb = test_13.size() / (1024 ** 2)
print(f"Size in w13: {size_mb:.2f} MB")
size_mb = test_14.size() / (1024 ** 2)
print(f"Size in w14: {size_mb:.2f} MB")

file:///sdf/group/rubin/repo/main/LSSTComCam/runs/DRP/DP1/w_2025_11/DM-49472/20250317T052612Z/object_forced_source/10704/22/object_forced_source_10704_22_lsst_cells_v1_LSSTComCam_runs_DRP_DP1_w_2025_11_DM-49472_20250317T052612Z.parq
file:///sdf/group/rubin/repo/main/LSSTComCam/runs/DRP/DP1/w_2025_11/DM-49472/20250317T052612Z/object_forced_source/10704/22/object_forced_source_10704_22_lsst_cells_v1_LSSTComCam_runs_DRP_DP1_w_2025_11_DM-49472_20250317T052612Z.parq
file:///sdf/group/rubin/repo/main/LSSTComCam/runs/DRP/DP1/w_2025_13/DM-49751/20250331T004753Z/object_forced_source/10704/22/object_forced_source_10704_22_lsst_cells_v1_LSSTComCam_runs_DRP_DP1_w_2025_13_DM-49751_20250331T004753Z.parq
file:///sdf/group/rubin/repo/main/LSSTComCam/runs/DRP/DP1/w_2025_14/DM-49864/20250406T123755Z/object_forced_source/10704/22/object_forced_source_10704_22_lsst_cells_v1_LSSTComCam_runs_DRP_DP1_w_2025_14_DM-49864_20250406T123755Z.parq
Size in w11: 18.38 MB
Size in w12: 18.38 MB
Size in w13: 3.10 MB
Siz

In [2]:
import pyarrow.parquet as pq
def compressions(file_name):
    compression_codecs = set([])
    parquet_metadata = pq.ParquetFile(file_name).metadata
    for i in range(parquet_metadata.num_row_groups):
        for j in range(parquet_metadata.num_columns):
            compression_codecs.add(parquet_metadata.row_group(i).column(j).compression)
    return compression_codecs



In [3]:
compressions(str(test_13)[7:])

{'SNAPPY'}

In [4]:
t13 = str(test_13)[7:]

In [5]:
tbl = pq.read_table(t13)

In [6]:
%cd ../rubin-user

/sdf/data/rubin/user/dtj1s


In [7]:
pq.write_table(tbl, './t13_zstd.pq', compression='ZSTD')

In [8]:
(3.1/2.5)-1

0.24

In [9]:
pq.write_table(tbl, './t13_brotli.pq', compression='BROTLI')

In [10]:
(2.5/2.4)-1

0.04166666666666674

In [11]:
parquet_file_name = str(test_12)[7:]
import time
import pyarrow.parquet as pq
import pandas as pd

table = pq.read_table(parquet_file_name)
output_dir = "."
compression_settings = [
    ('SNAPPY', None),
    ('ZSTD', 1),
    ('ZSTD', 3),
    ('ZSTD', 6),
    ('ZSTD', 9),
    ('ZSTD', 15),
]

results = []

for codec, level in compression_settings:
    filename = f"{output_dir}/sample_{codec.lower()}"
    if level is not None:
        filename += f"_lvl{level}"
    filename += ".parquet"
    
    # Write
    start_write = time.time()
    pq.write_table(
        table,
        filename,
        compression=codec,
        compression_level=level
    )
    write_time = time.time() - start_write
    
    # Read
    start_read = time.time()
    _ = pq.read_table(filename)
    read_time = time.time() - start_read

    # File size
    file_size = os.path.getsize(filename) / 1024**2  # MiB

    # Save results
    results.append({
        'Compression': codec,
        'Level': level if level is not None else '-',
        'File Size (MiB)': round(file_size, 1),
        'Write Time (s)': round(write_time, 4),
        'Read Time (s)': round(read_time, 4),
        'File': filename
    })

# 4. Display Results
results_df = pd.DataFrame(results)
results_df.sort_values(by='File Size (MiB)', inplace=True)
results_df.reset_index(drop=True, inplace=True)
results_df

,Compression,Level,File Size (MiB),Write Time (s),Read Time (s),File
0,ZSTD,15,15.7,0.5605,0.0400,./sample_zstd_lvl15.parquet
1,ZSTD,6,15.9,0.2668,0.0210,./sample_zstd_lvl6.parquet
2,ZSTD,9,15.9,0.3344,0.0185,./sample_zstd_lvl9.parquet
3,ZSTD,3,16.0,0.2175,0.0393,./sample_zstd_lvl3.parquet
4,ZSTD,1,16.1,0.1924,0.0342,./sample_zstd_lvl1.parquet
5,SNAPPY,-,18.4,0.1874,0.0406,./sample_snappy.parquet


In [12]:
snappy = results_df[results_df['Compression'] == 'SNAPPY'].iloc[0]
snappy

Compression                         SNAPPY
Level                                    -
File Size (MiB)                       18.4
Write Time (s)                      0.1874
Read Time (s)                       0.0406
File               ./sample_snappy.parquet
Name: 5, dtype: object

In [13]:
results_df['Size Win'] = snappy['File Size (MiB)'] / results_df['File Size (MiB)'] - 1
results_df['Time Win'] = snappy['Read Time (s)'] / results_df['Read Time (s)'] - 1
results_df

,Compression,Level,File Size (MiB),Write Time (s),Read Time (s),File,Size Win,Time Win
0,ZSTD,15,15.7,0.5605,0.0400,./sample_zstd_lvl15.parquet,0.171975,0.015000
1,ZSTD,6,15.9,0.2668,0.0210,./sample_zstd_lvl6.parquet,0.157233,0.933333
2,ZSTD,9,15.9,0.3344,0.0185,./sample_zstd_lvl9.parquet,0.157233,1.194595
3,ZSTD,3,16.0,0.2175,0.0393,./sample_zstd_lvl3.parquet,0.150000,0.033079
4,ZSTD,1,16.1,0.1924,0.0342,./sample_zstd_lvl1.parquet,0.142857,0.187135
5,SNAPPY,-,18.4,0.1874,0.0406,./sample_snappy.parquet,0.000000,0.000000


In [14]:
import time
import pyarrow.parquet as pq
import pandas as pd

table = pq.read_table(parquet_file_name)
output_dir = "."
compression_settings = [
    ('SNAPPY', None),
    ('GZIP', 1),
    ('GZIP', 3),
    ('GZIP', 6),
    ('GZIP', 9),
    ('ZSTD', 1),
    ('ZSTD', 3),
    ('ZSTD', 6),
    ('ZSTD', 9),
    ('ZSTD', 15),
]

results = []

for codec, level in compression_settings:
    filename = f"{output_dir}/sample_{codec.lower()}"
    if level is not None:
        filename += f"_{codec}_lvl{level}"
    filename += ".parquet"
    
    # Write
    start_write = time.time()
    pq.write_table(
        table,
        filename,
        compression=codec,
        compression_level=level
    )
    write_time = time.time() - start_write
    
    start_read = time.time()
    _ = pq.read_table(filename)
    read_time = time.time() - start_read

    file_size = os.path.getsize(filename) / 1024**2  # MiB

    results.append({
        'Compression': codec,
        'Level': level if level is not None else '-',
        'File Size (MiB)': round(file_size, 1),
        'Write Time (s)': round(write_time, 4),
        'Read Time (s)': round(read_time, 4),
        'File': filename
    })

results_df = pd.DataFrame(results)
results_df.sort_values(by='File Size (MiB)', inplace=True)
results_df.reset_index(drop=True, inplace=True)

snappy = results_df[results_df['Compression'] == 'SNAPPY'].iloc[0]
results_df['Size Win'] = snappy['File Size (MiB)'] / results_df['File Size (MiB)'] - 1
results_df['Time Win'] = snappy['Read Time (s)'] / results_df['Read Time (s)'] - 1
results_df

,Compression,Level,File Size (MiB),Write Time (s),Read Time (s),File,Size Win,Time Win
0,GZIP,6,15.6,1.7545,0.0541,./sample_gzip_GZIP_lvl6.parquet,0.179487,-0.293900
1,GZIP,9,15.7,1.2137,0.0487,./sample_gzip_GZIP_lvl9.parquet,0.171975,-0.215606
2,ZSTD,15,15.7,0.5482,0.0411,./sample_zstd_ZSTD_lvl15.parquet,0.171975,-0.070560
3,ZSTD,6,15.9,0.2661,0.0364,./sample_zstd_ZSTD_lvl6.parquet,0.157233,0.049451
4,ZSTD,9,15.9,0.2852,0.0412,./sample_zstd_ZSTD_lvl9.parquet,0.157233,-0.072816
5,ZSTD,3,16.0,0.2172,0.0164,./sample_zstd_ZSTD_lvl3.parquet,0.150000,1.329268
6,ZSTD,1,16.1,0.2063,0.0337,./sample_zstd_ZSTD_lvl1.parquet,0.142857,0.133531
7,GZIP,3,16.6,4.1067,0.1155,./sample_gzip_GZIP_lvl3.parquet,0.108434,-0.669264
8,GZIP,1,17.1,8.3794,0.0563,./sample_gzip_GZIP_lvl1.parquet,0.076023,-0.321492
9,SNAPPY,-,18.4,0.1789,0.0382,./sample_snappy.parquet,0.000000,0.000000


In [15]:
results_df.sort_values(by='Size Win')

,Compression,Level,File Size (MiB),Write Time (s),Read Time (s),File,Size Win,Time Win
9,SNAPPY,-,18.4,0.1789,0.0382,./sample_snappy.parquet,0.000000,0.000000
8,GZIP,1,17.1,8.3794,0.0563,./sample_gzip_GZIP_lvl1.parquet,0.076023,-0.321492
7,GZIP,3,16.6,4.1067,0.1155,./sample_gzip_GZIP_lvl3.parquet,0.108434,-0.669264
6,ZSTD,1,16.1,0.2063,0.0337,./sample_zstd_ZSTD_lvl1.parquet,0.142857,0.133531
5,ZSTD,3,16.0,0.2172,0.0164,./sample_zstd_ZSTD_lvl3.parquet,0.150000,1.329268
3,ZSTD,6,15.9,0.2661,0.0364,./sample_zstd_ZSTD_lvl6.parquet,0.157233,0.049451
4,ZSTD,9,15.9,0.2852,0.0412,./sample_zstd_ZSTD_lvl9.parquet,0.157233,-0.072816
1,GZIP,9,15.7,1.2137,0.0487,./sample_gzip_GZIP_lvl9.parquet,0.171975,-0.215606
2,ZSTD,15,15.7,0.5482,0.0411,./sample_zstd_ZSTD_lvl15.parquet,0.171975,-0.070560
0,GZIP,6,15.6,1.7545,0.0541,./sample_gzip_GZIP_lvl6.parquet,0.179487,-0.293900


In [16]:
results_df.sort_values(by='Time Win')

,Compression,Level,File Size (MiB),Write Time (s),Read Time (s),File,Size Win,Time Win
7,GZIP,3,16.6,4.1067,0.1155,./sample_gzip_GZIP_lvl3.parquet,0.108434,-0.669264
8,GZIP,1,17.1,8.3794,0.0563,./sample_gzip_GZIP_lvl1.parquet,0.076023,-0.321492
0,GZIP,6,15.6,1.7545,0.0541,./sample_gzip_GZIP_lvl6.parquet,0.179487,-0.293900
1,GZIP,9,15.7,1.2137,0.0487,./sample_gzip_GZIP_lvl9.parquet,0.171975,-0.215606
4,ZSTD,9,15.9,0.2852,0.0412,./sample_zstd_ZSTD_lvl9.parquet,0.157233,-0.072816
2,ZSTD,15,15.7,0.5482,0.0411,./sample_zstd_ZSTD_lvl15.parquet,0.171975,-0.070560
9,SNAPPY,-,18.4,0.1789,0.0382,./sample_snappy.parquet,0.000000,0.000000
3,ZSTD,6,15.9,0.2661,0.0364,./sample_zstd_ZSTD_lvl6.parquet,0.157233,0.049451
6,ZSTD,1,16.1,0.2063,0.0337,./sample_zstd_ZSTD_lvl1.parquet,0.142857,0.133531
5,ZSTD,3,16.0,0.2172,0.0164,./sample_zstd_ZSTD_lvl3.parquet,0.150000,1.329268
